In [1]:
# <div style="border:solid blue 2px; padding: 20px"> <h1 style="color:blue; margin-bottom:20px">Комментарий наставника</h1>

# Тут всё посчитано верно, молодец


# Задачи
Вы аналитик в онлайн-маркетплейсе «Заберу». Это настоящий рай для любителей бесполезных сувениров — там можно приобрести товары ручной работы от небольших производителей. «Заберу» проводит рекламные акции длительностью 1 день. Руководитель отдела маркетинга хочет узнать, насколько эффективны такие кампании.

## Цели исследования:
- Посчитайте затраты на каждый рекламный источник.
- Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов, сделанных пользователями, которые хотя бы раз перешли на сайт с настольных ПК (тип устройства desktop) из рекламного источника, на который было потрачено больше всего денег. Определить рекламный источник, на который было потрачено больше всего денег, можно как из результатов выполнения задания 1, так и с помощью подзапроса. Сделайте выводы.
- Введём новую метрику эффективности рекламны - дневные затраты на рекламу на пользователя по источнику. Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля

In [1]:
!pip install psycopg2-binary

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')

from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=False)

# Исследуем таблицы

## Таблица visits

In [3]:
query = '''
SELECT *
FROM visits
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,id,uid,device,endts,sourceid,startts
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00
1,1,104060357244891740,desktop,2018-02-19 17:21:00,2,2018-02-19 16:53:00
2,2,7459035603376831527,touch,2017-07-01 01:54:00,5,2017-07-01 01:54:00
3,3,16174680259334210214,desktop,2018-05-20 11:23:00,9,2018-05-20 10:59:00
4,4,9969694820036681168,desktop,2017-12-27 14:06:00,3,2017-12-27 14:06:00


---

## Таблица orders

In [4]:
query = '''
SELECT *
FROM orders
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844
3,4,2017-06-01 00:29:00,$0.55,16109239769442553005
4,5,2017-06-01 07:58:00,$0.37,14200605875248379450


---

## Таблица advertisment_costs

In [5]:
query = '''
SELECT *
FROM advertisment_costs
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53
3,4,1,2017-06-04,$55.00
4,5,1,2017-06-05,$57.08


---

# Сделаем запросы под задачи

## Посчитайте затраты на каждый рекламный источник.

In [6]:
query = '''
SELECT sourceid AS "Источник",
       SUM(costs) AS "Затраты"
FROM advertisment_costs
GROUP BY sourceid
'''
pd.io.sql.read_sql(query, con = engine)

,Источник,Затраты
0,10,"$5,822.49"
1,9,"$5,517.49"
2,1,"$20,833.27"
3,5,"$51,757.10"
4,4,"$61,073.60"
5,2,"$42,806.04"
6,3,"$141,321.63"


3 источник — самый затратный, 9 источник — менее затратный.

---

<div style="border:solid blue 2px; padding: 20px"> <h1 style="color:blue; margin-bottom:20px">Комментарий наставника</h1>

Тут всё посчитано верно, молодец


## Посчитайте суммарную выручку, максимальную и минимальную выручку от заказов

Речь, о сделанных пользователями, которые хотя бы раз перешли на сайт с настольных ПК (тип устройства desktop) из рекламного источника, на который было потрачено больше всего денег. Определить рекламный источник, на который было потрачено больше всего денег, можно как из результатов выполнения задания 1, так и с помощью подзапроса. Сделайте выводы.

In [7]:
query = '''
SELECT SUM(revenue) AS Итого,
       MAX(revenue) AS Максимум,
       MIN(revenue) AS Минимум
FROM orders
WHERE orders.revenue >= '0' and uid in
(
SELECT uid
FROM visits
WHERE visits.sourceid = 3 and device = 'desktop');
'''
pd.io.sql.read_sql(query, con = engine)

,Итого,Максимум,Минимум
0,"$106,058.86","$2,633.28",$0.00


Есть большой разлет между максимальным значением и минимальным, но это по сути ничего не говорит. Есть нулевые заказы — хотелось бы понять откуда они и что значат.

---

<div style="border:solid blue 2px; padding: 20px"> <h1 style="color:blue; margin-bottom:20px">Комментарий наставника</h1>

Результаты запроса правильные, супер!

## Введём новую метрику эффективности рекламны - дневные затраты на рекламу на пользователя по источнику.

Она рассчитывается для каждого отдельного дня и равна объёму денежных средств, которые были израсходованы на то, чтобы пользователь перешёл по рекламе на сайт из данного рекламного источника. Иными словами, метрика для каждого дня равна отношению рекламных затрат по данному источнику к количеству посетителей, перешедших на сайт с данного источника. Рассчитайте для каждого дня введённую метрику. Считайте, что целевое действие было произведено тогда, когда сесия началась. Например, если посетитель провёл на сайте несколько минут с 23:50 14 апреля по 0:20 15 апреля, то его нужно включать в рассчёт за 14 апреля

In [8]:
query = '''
WITH
    costs_sum AS(
SELECT
    dt,
    sourceid,
    SUM(costs) AS Расходы
FROM
    advertisment_costs
GROUP BY
    dt,
    sourceid),
    visits_unique AS(
    SELECT
    visits.sourceid,
    DATE_TRUNC('DAY', visits.startts) AS date,
    COUNT(DISTINCT(visits.uid)) AS count_unique_users        
FROM
    visits
GROUP BY
    DATE_TRUNC('DAY', visits.startts),
    visits.sourceid)
          SELECT
                Расходы,
                dt AS Дата,
                visits_unique.sourceid AS Источник,
                Расходы / count_unique_users AS Метрика
          FROM
                visits_unique
          RIGHT JOIN costs_sum ON costs_sum.dt = visits_unique.date AND
                               costs_sum.sourceid = visits_unique.sourceid;
'''
temp = pd.io.sql.read_sql(query, con = engine)
pd.io.sql.read_sql(query, con = engine)

,Расходы,Дата,Источник,Метрика
0,$75.20,2017-06-01,1.0,$1.27
1,$132.56,2017-06-01,2.0,$1.67
2,$349.38,2017-06-01,3.0,$1.91
3,$76.74,2017-06-01,4.0,$0.55
4,$73.24,2017-06-01,5.0,$0.64
...,...,...,...,...
2537,$492.54,2018-05-31,4.0,$0.98
2538,$398.04,2018-05-31,5.0,$1.69
2539,$27.37,2018-05-31,9.0,$0.44
2540,$17.60,2018-05-31,10.0,$0.14


<div style="border:solid blue 2px; padding: 20px"> <h1 style="color:blue; margin-bottom:20px">Комментарий наставника</h1>
Задание выполнено не верно, тут требуется найти затраты на каждого пользователя

В таких сложных запросах хорошо делать проверку, для этого нужно вывести еще столбцы расходов и количества посетителей. Кроме того, нужно посчитать сумму по столбцу расходов - она должна совпасть с суммой расходов из таблицы advertisment_costs.

In [9]:
temp['Расходы'] = temp['Расходы'].str.replace('$', '')
temp.loc[:,'Расходы'] = temp.loc[:,'Расходы'].str.replace(',', '', regex = True)
temp['Расходы'] = temp['Расходы'].astype(float)

In [10]:
temp['Расходы'].sum()

329131.62

<div style="border:solid blue 2px; padding: 20px"> <h1 style="color:blue; margin-bottom:20px">Комментарий наставника</h1>

Результаты верные, замечательно!

---